In [1]:
from bs4 import BeautifulSoup
import requests
from db import PythonProblems
from datetime import datetime
from urllib.parse import urljoin
from tqdm import tqdm

### Creating and connecting to database

In [2]:
db = PythonProblems('python.sqlite', 'init_tables.sql')

### Getting the pages to be crawled

In [30]:
%%time
urls = [
       # "https://www.w3resource.com/python-exercises/python-basic-exercises.php", 
       # "https://www.w3resource.com/python-exercises/basic/", # commit d66ecae4b3ee871ca6d94d90ac9fde7d28dd1c67
       # "https://www.w3resource.com/python-exercises/string/",
       # "https://www.w3resource.com/python-exercises/list/", # commit 42e8eb9a78d7474cecf2c373e3c9c9b4652423e7
       # "https://www.w3resource.com/python-exercises/dictionary/", # commit 6490b5779a9f472edbe7cd286d579557b2b53411
       # "https://www.w3resource.com/python-exercises/tuple/", # commit aa04882c57893437f896b46e755433a64b34c1c6
       # "https://www.w3resource.com/python-exercises/sets/", # commit ebd13e285ccaa3437b6b7076d01b18d5f9c06277
       # "https://www.w3resource.com/python-exercises/array/", # commit 5d43e87ca2614bc42b11a3ada50616c48eac5dd8
       # "https://www.w3resource.com/python-exercises/python-conditional-statements-and-loop-exercises.php", # commit b03706f3a3d218675c73c7457e345f3158d5f8df
       # "https://www.w3resource.com/python-exercises/python-functions-exercises.php", # commit 79c06e3600b544d31e1995c65f6a76559838a5fb
        "https://www.w3resource.com/python-exercises/math/index.php"]
pms = []
for idx, url in enumerate(urls):
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html5lib')
    solution_links, problem_comparison, problems = get_problems(soup)
    
    #if idx == 0:
    #    # Bug-fix, problem is written wrong in the first page
    #    problems[145]["content"] = "Write a Python program to test if a variable is a list or tuple or a set."
    
    #if idx == 5:
    #    # Bug-fix, problem is written wrong in the first page
    #    problems[21]["content"] = "Write a Python program to remove an empty tuple(s) from a list of tuples."
    
    if idx == 8:
    #if idx == 0:
        # Bug-fix, problem is written wrong in the first page
        problems[25]['content'] = "Write a Python program to print alphabet pattern 'S'"
        problems.insert(26, problems[26].copy())
        problems[26]['content'] = "Write a Python program to print alphabet pattern 'T'"
        solution_links = []
        aTag = soup.find_all('a')
        for a in aTag:
            if a.text.strip() == "Click me to see the sample solution":
                solution_links.append(a)
        problem_comparison.insert(26, "")
        
        problems[33]['content'] = "Write a Python program to sum of two given integers. However, if the sum is between 15 to 20 it will return 20."
        
    #if idx == 9:
    if idx == 0:
        problems[30]['content'] = "Write a Python program to convert a binary number to decimal number."
        problems[34]['content'] = "Write a Python program to convert Polar coordinates to rectangular coordinates."
    
    # Dictionary has problem repetition, but different solutions
    if idx == 4:
        solutions, potential_mismatch = get_solutions(solution_links, problem_comparison, problems)
    else:
        solutions, potential_mismatch = get_solutions(solution_links, problem_comparison)
        
    pms.append(potential_mismatch)
    insert(problems, solutions)
#db.close_connection()

100%|██████████| 84/84 [00:00<00:00, 6755.80it/s]
82it [01:07,  1.17it/s]

Got 82 problems inserted, 82 solutions inserted, 0 problems repeated and 0 solutions repeated
CPU times: user 6.25 s, sys: 272 ms, total: 6.52 s
Wall time: 1min 9s


## Functions

### Problems and solutions links

In [3]:
def get_problems(soup):
    exercises = soup.find_all("strong")
    problem_list = []
    solution_links = []
    problem_comparison = []
    
    # First replace <br> with line break
    for br in soup.find_all("br"):
        br.replace_with("\n")

    title = soup.find_all("h2", class_="heading")[0].text.split("[")[0].strip()

    for e in tqdm(exercises):
        # Find solutions links
        solution_link = e.findNext("a")
        try:
            while solution_link.text.strip() != "Click me to see the sample solution":
                solution_link = solution_link.findNext("a")
        except AttributeError:
            continue
        solution_links.append(solution_link)

        # Get problem
        problem_text = ""
        problem_text = e.parent.text
        nextTag = e.parent.next_sibling
        while not isinstance(nextTag, type(e)):
            nextTag = nextTag.next_sibling
        if nextTag and nextTag.name == "pre":
            problem_text += nextTag.text

        # Get original text for comparison in solution's page
        editor_partition = problem_text.partition("Go to the editor")
        #Go to the editor is written
        if editor_partition[2] != '':
            problem_comparison.append(editor_partition[0].split('.')[1].strip())
        # Go to the editor is not written
        else:
            problem_comparison.append(problem_text.split('.')[1])

        problem_text = problem_text.replace("Go to the editor", "")
        problem_text = problem_text.replace("Click me to see the sample solution", "")
        problem_text = problem_text.partition('.')[2].strip()
        problem_list.append(problem_text)

    problems = []

    for problem_text in problem_list:
        problems.append({"title": title, "link": url,
                         "content": problem_text, "retrieved_date": datetime.now(),
                         "crawler": "W3Resource"})
        
    return solution_links, problem_comparison, problems

### Solutions

In [4]:
def get_solutions(solution_links, problem_comparison, problems=None):
    solutions = []
    potential_mismatch = []
    
    if problems:
        problems_list = [p['content'] for p in problems]
    
    for idx, link in tqdm(enumerate(solution_links)):
        link_sol = urljoin(url, link['href'])
        response = requests.get(link_sol)
        soup = BeautifulSoup(response.text, 'html5lib')

        # Compare problem inside with original problem
        compare_problem = soup.find_all("h2")[0].findNext().text
        if compare_problem[:-1].strip() != problem_comparison[idx].strip():
            potential_mismatch.append(link_sol + '\n' + compare_problem[:-1].strip() + '\n' + problem_comparison[idx] + '\n\n')

        code = soup.find_all("code")
        
        if problems:
            idx_sol = problems_list.index(problems_list[idx])
        else:
            idx_sol = idx
        solutions.append({"content": code[0].text, "link": link_sol,
                          "retrieved_date": datetime.now(), "ignore": False,
                          "idx": idx_sol})
    return solutions, potential_mismatch

### Insert rows

In [5]:
def insert(problems, solutions):
    db.insert_rows(problems, solutions)
    tp, ts, rp, rs = db.populate()
    print("Got %d problems inserted, %d solutions inserted, %d problems repeated and %d solutions repeated" % (
          tp, ts, rp, rs))

### Control check

In [27]:
for item in pms[0]:
    print(item)

https://www.w3resource.com/python-exercises/math/python-math-exercise-1.php
Write a Python program to convert degree to radian.
Note : The radian is the standard unit of angular measure, used in many areas of mathematics. An angle's measurement in radians is numerically equal to the length of a corresponding arc of a unit circle; one radian is just under 57.3 degrees (when the arc length is equal to the radius)
Write a Python program to convert degree to radian


https://www.w3resource.com/python-exercises/math/python-math-exercise-2.php
Write a Python program to convert radian to degree.
Note: The radian is the standard unit of angular measure, used in many areas of mathematics. An angle's measurement in radians is numerically equal to the length of a corresponding arc of a unit circle; one radian is just under 57.3 degrees (when the arc length is equal to the radius)
Write a Python program to convert radian to degree


https://www.w3resource.com/python-exercises/math/python-math-exer

In [25]:
db.rows

[{'problem': {'content': 'Write a Python function to find the Max of three numbers.',
   'crawler': 'W3Resource',
   'link': 'https://www.w3resource.com/python-exercises/python-functions-exercises.php',
   'retrieved_date': datetime.datetime(2017, 12, 12, 18, 34, 12, 54707),
   'title': 'Python functions'},
  'solution': {'content': 'def max_of_two( x, y ):\n    if x > y:\n        return x\n    return y\ndef max_of_three( x, y, z ):\n    return max_of_two( x, max_of_two( y, z ) )\nprint(max_of_three(3, 6, -5))\n',
   'ignore': False,
   'link': 'https://www.w3resource.com/python-exercises/python-functions-exercise-1.php',
   'problem_id': 622,
   'retrieved_date': datetime.datetime(2017, 12, 12, 18, 34, 13, 122865)}},
 {'problem': {'content': 'Write a Python function to sum all the numbers in a list. \n\nSample List : (8, 2, 3, 0, 7)\n\nExpected Output : 20',
   'crawler': 'W3Resource',
   'link': 'https://www.w3resource.com/python-exercises/python-functions-exercises.php',
   'retriev